<a href="https://colab.research.google.com/github/beercafeguy/PyMLLab/blob/master/ML_A_Z/regression/regression_model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path = '/content/drive/MyDrive/ML_A_Z/Model_Selection/Regression/Data.csv'

In [10]:
df = pd.read_csv(path)
df
#PE  - dependent variable
# rest independent variable

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90
...,...,...,...,...,...
9563,16.65,49.69,1014.01,91.00,460.03
9564,13.19,39.18,1023.67,66.78,469.62
9565,31.32,74.33,1012.92,36.48,429.57
9566,24.48,69.45,1013.86,62.39,435.74


In [11]:
df.isnull().sum()

AT    0
V     0
AP    0
RH    0
PE    0
dtype: int64

In [12]:
df.shape

(9568, 5)

In [13]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [14]:
X

array([[  14.96,   41.76, 1024.07,   73.17],
       [  25.18,   62.96, 1020.04,   59.08],
       [   5.11,   39.4 , 1012.16,   92.14],
       ...,
       [  31.32,   74.33, 1012.92,   36.48],
       [  24.48,   69.45, 1013.86,   62.39],
       [  21.6 ,   62.52, 1017.23,   67.87]])

In [15]:
y

array([463.26, 444.37, 488.56, ..., 429.57, 435.74, 453.28])

In [16]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=123,test_size=0.2)

## Multiple Linear Regression

In [17]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [19]:
y_pred = lr.predict(X_test)
np.set_printoptions(precision=2)
print(
    np.concatenate(
        (y_pred.reshape(len(y_pred),1),
        y_test.reshape(len(y_pred),1)),
        1
        )
    )

[[447.85 455.41]
 [437.9  441.64]
 [471.87 471.63]
 ...
 [437.65 430.2 ]
 [444.62 441.58]
 [486.41 493.72]]


In [20]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.9235430062617527

## Polinomial Regression

In [21]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_feat = PolynomialFeatures(degree = 4)
X_poly = poly_feat.fit_transform(X_train)
plr = LinearRegression()
plr.fit(X_poly,y_train)

LinearRegression()

In [22]:
y_pred = plr.predict(poly_feat.transform(X_test))
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[449.78 455.41]
 [440.66 441.64]
 [468.29 471.63]
 ...
 [436.5  430.2 ]
 [445.14 441.58]
 [488.11 493.72]]


In [23]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.9391443793443689

## SVM Regressor

In [30]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train_scaled = sc_X.fit_transform(X_train)
y_train_scaled = sc_y.fit_transform(y_train.reshape(len(y_train),1))

In [31]:
X_train_scaled

array([[-0.09, -0.95, -0.04,  0.41],
       [ 0.48, -0.45, -1.82, -0.84],
       [ 0.32,  0.36, -0.39,  0.92],
       ...,
       [-1.57, -1.24,  0.55, -0.3 ],
       [-0.07,  0.18,  0.39,  1.32],
       [ 0.85,  0.39, -0.39, -0.2 ]])

In [32]:
#y_train_scaled = y_train_scaled.reshape(1,len(y_train_scaled))
y_train_scaled

array([[ 0.49],
       [-0.52],
       [-0.33],
       ...,
       [ 1.84],
       [-0.3 ],
       [-0.6 ]])

In [33]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train_scaled, y_train_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

In [34]:
y_pred = sc_y.inverse_transform(regressor.predict(sc_X.transform(X_test)).reshape(-1,1))
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[451.48 455.41]
 [441.   441.64]
 [468.98 471.63]
 ...
 [436.02 430.2 ]
 [444.54 441.58]
 [488.15 493.72]]


In [35]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.941079687764569

## Decision Tree Regression

In [36]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [37]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[458.31 455.41]
 [438.25 441.64]
 [468.54 471.63]
 ...
 [434.47 430.2 ]
 [442.02 441.58]
 [492.96 493.72]]


In [38]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.9259700127294458

## Random Forest Regression

In [39]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=0)

In [40]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[455.55 455.41]
 [440.23 441.64]
 [469.33 471.63]
 ...
 [436.21 430.2 ]
 [442.51 441.58]
 [491.54 493.72]]


In [41]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.9567452112032203